In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Program, Task
import pandas as pd
from dataclasses import replace

from code_widget.example import CodeWidget
import json

In [2]:
# CHANGE ME!
TASK_ID = 'scc'
AUTHOR = 'scott'

In [36]:
task = Task(
    id=TASK_ID,
    description="Given a graph and a vertex v, list the vertices in the strongly-connected component of v.",
    plan=[{
        "id": "reachability",
        "description": "get TC from edge relation",
    }, {
        "id": "scc",
        "description": "get mutually reachable vertices",
    }, {
        "id": "query",
        "description": "list SCC vertices from the query vertex",
    }
    ],
    sample_input={
        "graph": [
            {"source": "a", "target": "b"},
            {"source": "b", "target": "b"},
            {"source": "b", "target": "c"},
            {"source": "c", "target": "b"},
            {"source": "c", "target": "a"},
            {"source": "a", "target": "d"},
        ],
        "query": [{"source": "c"}]
    },
    sample_output=["a", "b", "c"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [6]:
program = datalog
# widget = CodeWidget(program=program.to_json(), task=task.to_json())
# widget
program.widget(task)

Output()

CodeWidget(program='{"task": "scc", "language": "datalog", "plan": {}, "source": "\\n.decl scc1(x: symbol, y: …

In [82]:
# new_program = replace(program, plan=json.loads(widget.plans))
# new_program.save()

In [12]:
datalog = replace(prototype,
    language='datalog',
    source='''
.decl scc1(x: symbol, y: symbol)
.decl path(x: symbol, y: symbol)
.decl vertex(x: symbol)

scc(x) :- query(src), scc1(src, x).
scc1(x, x) :- vertex(x).
scc1(x, y) :- path(x, y), path(y, x).
path(x, y) :- graph(x, y).
path(x, y) :- graph(x, z), path(z, y).
vertex(x)  :- graph(x, _).
vertex(x)  :- graph(_, x).
''').load_plan()
datalog.execute(task)
datalog.save()

In [38]:
sql = replace(prototype,
    language='sql',
    source='''
WITH RECURSIVE
closure(source, target) AS (
  SELECT DISTINCT source, target FROM graph
  UNION
  SELECT edge.source, path.target
  FROM closure as path JOIN graph as edge
  ON edge.target = path.source
),
component(v1, v2) AS (
  SELECT path.source, path.target
  FROM closure as path JOIN closure as path_
  ON path.source = path_.target AND path.target = path_.source
)
SELECT S.v2 FROM component as S
JOIN query ON S.v1 = query.source
''').load_plan()
sql.execute(task)
sql.save()

In [4]:
pandas = replace(prototype,
    language='python-pandas',
    implementation='',
    source='''
import pandas as pd
def find_uncommon_words(documents):
  return documents[0]["text"].split(" ")[0]
''')
pandas.execute(task)
pandas.save()

In [39]:
python_imp = replace(prototype,
    language='python-imperative',
    implementation='',
    source='''
def scc(graph, query):
    reachable = set()
    vertices = set()
    for edge in graph:
        reachable.add((edge["source"], edge["target"]))
        vertices.add(edge["source"])
        vertices.add(edge["target"])
    changed = True
    while changed:
        changed = False
        for edge in graph:
            for vertex in vertices:
                if (edge["source"], vertex) not in reachable and (edge["target"], vertex) in reachable:
                    changed = True
                    reachable.add((edge["source"], vertex))
    source = query[0]["source"]
    result = []
    for vertex in vertices:
        if (source, vertex) in reachable and (vertex, source) in reachable:
            result.append(vertex)
    return result
''')
python_imp.execute(task)
python_imp.save()

In [46]:
def scc(graph, query):
    reachable = set()
    vertices = set()
    for edge in graph:
        reachable.add((edge["source"], edge["target"]))
        vertices.add(edge["source"])
        vertices.add(edge["target"])
    changed = True
    while changed:
        changed = False
        for edge in graph:
            for vertex in vertices:
                if (edge["source"], vertex) not in reachable and (edge["target"], vertex) in reachable:
                    changed = True
                    reachable.add((edge["source"], vertex))
    source = query["source"]
    result = []
    for vertex in vertices:
        if (source, vertex) in reachable and (vertex, source) in reachable:
            result.append(vertex)
    return result

In [40]:
# todo: pandas. maybe: make python-imperative more idiomatic
# not sure if a separate python-functional case makes sense